# Custom activation funcs, initializers, regualizers and constraints

## Importing libs

In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.Useful_funcs import data_pipeline, compile_train, pre_model # Custom funcs for data processing, modelling, compiling and training
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import selu
from tensorflow.keras.initializers import lecun_normal
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import mse

## Loading data

In [2]:
housing = fetch_california_housing()
x_train, x_train_scaled, x_valid, x_valid_scaled, x_test, x_test_scaled, y_train, y_valid, y_test = data_pipeline(housing)

In [3]:
x_train.shape

(11610, 8)

In [4]:
x_train_scaled.shape

(11610, 8)

## Model

- Below are some of the custom func that we can use in the models. Even though these are readily available in the high-level API, we will be using these custom funcs.

In [5]:
# Softplus activation func
def softplus(z):
    return tf.math.log(tf.exp(z) + 1)

In [6]:
# Glorot initializers
def glorot_initializer(shape, dtype = tf.float32):
    std = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev = std, dtype = dtype)

In [7]:
# l1 regularization
def l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [8]:
# Constraint func to keep the weights positive
def positive_weights(weights):
    return tf.where(weights < 0, tf.zeros_like(weights), weights)

- The above custom funcs can be used in a layer as follows

In [9]:
layer = Dense(1, activation = softplus, kernel_initializer = glorot_initializer, kernel_regularizer = l1_regularizer, kernel_constraint = positive_weights)

In [10]:
pre_model()

In [11]:
input_shape = x_train.shape[1:]

In [12]:
# Building the model
model = Sequential()
model.add(Dense(30, activation = selu, kernel_initializer = lecun_normal(), input_shape = input_shape))
model.add(Dense(1, activation = softplus, kernel_initializer = glorot_initializer, kernel_regularizer = l1_regularizer, kernel_constraint = positive_weights))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compiling and training the model
history = compile_train(model, (x_train_scaled, y_train), (x_valid_scaled, y_valid), loss = mse, optimizer = Nadam(), metrics = 'mae', epochs = 5)

Epoch 1/5
363/363 [==============================] - 1s 1ms/step - loss: 1.5542 - mae: 0.8962 - val_loss: 1.4154 - val_mae: 0.5607
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 0.5943 - mae: 0.5256 - val_loss: 1.4399 - val_mae: 0.5137
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 0.5260 - mae: 0.4968 - val_loss: 1.0372 - val_mae: 0.4885
Epoch 4/5
363/363 [==============================] - 0s 997us/step - loss: 0.5008 - mae: 0.4879 - val_loss: 0.8910 - val_mae: 0.4820
Epoch 5/5
363/363 [==============================] - 0s 1ms/step - loss: 0.4910 - mae: 0.4831 - val_loss: 0.7684 - val_mae: 0.4791


In [15]:
model.save('Custom_func_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Custom_func_model/assets


In [16]:
model = keras.models.load_model('Custom_func_model', 
                                custom_objects = {'softplus' : softplus, 'glorot_initializer' : glorot_initializer, 
                                                  'l1_regularizer' : l1_regularizer, 'positive_weights' : positive_weights})

- If we have to sava the hyperparameter of a custom model along with the model, then subclass the appropriate base class.

In [17]:
# Subclassing regularizer class of keras
class l1_regularization(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(weights * self.factor))
    def get_config(self):
        return {'factor' : self.factor}

In [19]:
pre_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [20]:
model = Sequential()
model.add(Dense(30, activation = selu, kernel_initializer = lecun_normal(), input_shape = input_shape))
model.add(Dense(1, activation = softplus, kernel_initializer = glorot_initializer, kernel_regularizer = l1_regularization(0.01), kernel_constraint = positive_weights))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = compile_train(model, (x_train_scaled, y_train), (x_valid_scaled, y_valid), loss = mse, optimizer = Nadam(), metrics = 'mae', epochs = 5)

Epoch 1/5
363/363 [==============================] - 0s 1ms/step - loss: 1.5542 - mae: 0.8962 - val_loss: 1.4154 - val_mae: 0.5607
Epoch 2/5
363/363 [==============================] - 0s 935us/step - loss: 0.5943 - mae: 0.5256 - val_loss: 1.4399 - val_mae: 0.5137
Epoch 3/5
363/363 [==============================] - 0s 947us/step - loss: 0.5260 - mae: 0.4968 - val_loss: 1.0372 - val_mae: 0.4885
Epoch 4/5
363/363 [==============================] - 0s 961us/step - loss: 0.5008 - mae: 0.4879 - val_loss: 0.8910 - val_mae: 0.4820
Epoch 5/5
363/363 [==============================] - 0s 982us/step - loss: 0.4910 - mae: 0.4831 - val_loss: 0.7684 - val_mae: 0.4791


In [23]:
model.save('Custom_func_model')

INFO:tensorflow:Assets written to: Custom_func_model/assets


In [24]:
model = keras.models.load_model('Custom_func_model', 
                                custom_objects = {'softplus' : softplus, 'glorot_initializer' : glorot_initializer, 
                                                  'l1_regularization' : l1_regularization, 'positive_weights' : positive_weights})